In [30]:
import pandas as pd

In [31]:
TABLES = ['case_data','customer_data', 'insurance_case_data','payout_data']
DATA_FOLDER = 'data'
# creating dataframe names dynamically is typically a bad practice, hardcode the names for now
df_case_data = pd.read_csv(f'{DATA_FOLDER}/case_data.csv')


In [32]:
CLEANED_DATA_FOLDER = 'cleaned_data'
import os
try:
    os.makedirs(f'{CLEANED_DATA_FOLDER}/')
except:
    pass

In [33]:
import json

def replace_strings(column, replace_dicts, regex=True):
    """Replaces strings in text for a column of dicts

    Args:
        column (df): column of a dataframe
        replace_dicts (dict): dictionary
        regex (bool, optional): Determines if the passed-in pattern is a regular expression. Defaults to True.

    Returns:
        df: column with strings updated
    """
    new_col = column.copy()
    for old, new in replace_dicts:
        new_col = new_col.str.replace(old, new, regex=regex)
    return new_col


In [34]:
def flatten_objects(df):
    extracted_objects = pd.json_normalize(replace_strings(df.payload, [
        ("True", "true"),
        ("None", "null"),
        ("False", "false"),
        ("'", '"')
    ]).apply(json.loads))
    df_output = pd.concat([df,extracted_objects], axis=1)
    del df_output['payload']
    
    return df_output

In [35]:
tables_to_flatten = ['customer_data','insurance_case_data','payout_data']
for table in tables_to_flatten:
    df = pd.read_csv(f'{DATA_FOLDER}/{table}.csv')
    flattened_df = flatten_objects(df)
    flattened_df.to_csv(f'{CLEANED_DATA_FOLDER}/{table}.csv', index = False)


In [36]:
# Split out the case data table into normalized tables
def normalize_object_data(df,objects_to_extract):
    for data_point in objects_to_extract:
        payload_dict = replace_strings(df.payload, [
            ("True", "true"),
            ("None", "null"),
            ("False", "false"),
            ("'", '"')
        ]).apply(json.loads)

        payload_data = payload_dict.apply(lambda x: x[f'{data_point}'])
        df_data = pd.concat([df['case_id'],pd.json_normalize(payload_data)], axis=1)
        df_data.to_csv(f'{CLEANED_DATA_FOLDER}/{data_point}.csv', index = False)
# Will only normalize data for df_case_data
objects_to_normalize = ['person','incident','occupation']
normalize_object_data(df_case_data,objects_to_normalize)

In [37]:
del df_case_data['payload']
df_case_data.to_csv(f'{CLEANED_DATA_FOLDER}/case_data.csv', index = False)
